# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [27]:
# import libraries
import sqlite3
import pandas as pd
import re
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster_ETL.db')
df = pd.read_sql_table('Categories', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df['message'] #target variable
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1) #all others are features (categories to predict)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    #Normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    #Tokenize
    tokens = word_tokenize(text)
    
    #stemmed = [PorterStemmer().stem(w) for w in tokens]
    
    #Set stopword idiom
    stop_words = stopwords.words("english")
    # Reduce words to their root form and remove stop words
    clean_text = [WordNetLemmatizer().lemmatize(word) for word in tokens if word not in stop_words]
    
    return clean_text

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
# Pipeline: RandomForestClassifier classifier
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
#split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# train pipeline classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# pipline prediction
Y_pred = pipeline.predict(X_test)
Y_pred.shape

(7865, 36)

In [12]:
def model_report(prediction, X_test, y_test):
    '''
    Function to generate classification report on the model
    prediction: predicted model
    x_test: target test 
    y_test: features test
    Output: prints a classification model report by categories
    '''
    for i, col in enumerate(y_test):
        print(col)
        print("------------------------------------------------------")
        print(classification_report(y_test[col], prediction[:, i]))

In [13]:
# pipeline predict
Y_pred = pipeline.predict(X_test)

#print the results report 
model_report(Y_pred, X_test, y_test)

related
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.62      0.46      0.53      1873
          1       0.84      0.90      0.87      5934
          2       0.24      0.34      0.28        58

avg / total       0.78      0.79      0.78      7865

request
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      6533
          1       0.78      0.42      0.55      1332

avg / total       0.87      0.88      0.87      7865

offer
------------------------------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

aid_related
------------------------------------------------------
             precision    recall  f1-score   sup

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# pipeline params
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7feb80289e18>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [15]:
#parameters = {'vect__min_df': [1, 5],
 #             'tfidf__use_idf':[True, False],
 #             'clf__estimator__n_estimators':[10, 25], 
 #             'clf__estimator__min_samples_split':[2, 5, 10]}

# parameters
parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [10, 20]}
             # 'clf__estimator__min_samples_split': [2, 4]}


In [16]:
# get and fit GridSearchCV
cv = GridSearchCV(pipeline, param_grid=parameters)
tunned_ = cv.fit(X_train, y_train)

In [17]:
#tunned_.cv_results_

In [18]:
# best tunned_ params 
print(tunned_.best_params_)   
#best tunned_ score
print(tunned_.best_score_)

{'clf__estimator__n_estimators': 20, 'tfidf__use_idf': True}
0.249904637349


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
# Test the model
Y_pred_tunned = cv.predict(X_test)

(7865, 36)

In [20]:
#print the results report 
model_report(Y_pred_tunned, X_test, y_test)

related
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.67      0.43      0.53      1873
          1       0.84      0.93      0.88      5934
          2       0.37      0.33      0.35        58

avg / total       0.79      0.81      0.79      7865

request
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      6533
          1       0.80      0.46      0.59      1332

avg / total       0.88      0.89      0.88      7865

offer
------------------------------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

aid_related
------------------------------------------------------
             precision    recall  f1-score   sup

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
# Improving the pipeline: AdaBoostClassifier classifier
pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [22]:
# pipeline params
ppeline_2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7feb80289e18>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [23]:
parameters2 = { #'vect__ngram_range': ((1, 1), (1, 2)), 
              #'vect__max_df': (0.5, 1.0), 
              #'vect__max_features': (None, 5000), 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [10, 10],
              'clf__estimator__learning_rate': [1,2]}
               

In [24]:
# improving and fit GridSearchCV with new parameters 
cv_2 = GridSearchCV(pipeline_2, param_grid=parameters2)
tunned_2 = cv_2.fit(X_train, y_train)

In [25]:
# predict and report the results
Y_pred_tunned_2 = cv_2.predict(X_test)
model_report(Y_pred_tunned_2, X_test, y_test)

related
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1873
          1       0.75      1.00      0.86      5934
          2       0.67      0.03      0.07        58

avg / total       0.57      0.75      0.65      7865

request
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      6533
          1       0.75      0.44      0.55      1332

avg / total       0.87      0.88      0.87      7865

offer
------------------------------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

aid_related
------------------------------------------------------
             precision    recall  f1-score   sup

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [28]:
# Export the model
pickle.dump(cv_2, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.